In [4]:
import pandas as pd
import openpyxl
import xlrd

In [5]:
tarjetas = pd.read_excel('CASHFLOW Conbra nov 22.xls')
bancos = pd.read_excel('localesybancos2.xlsx')
%store bancos

Stored 'bancos' (DataFrame)


In [6]:
tarjetas

,Fecha de Presentacion,Fecha de Pago,Cod Concepto Clearing,Desc Concepto Clearing,Nro Liquidacion,Cod Sucursal Pagadora,Cod Comercio,Nombre Comercio,Cod Entidad Pagadora,Marca Producto,Importe Concepto,Cuit Ruc Num
0,2022-10-04,2022-11-01,1009,VENTAS C/DESCUENTO CONTADO,144402,506,4547438,LE UTTHE,16,Mastercard,105333.25,30602291487
1,2022-10-04,2022-11-01,1009,VENTAS C/DESCUENTO CONTADO,144450,506,25107174,LE UTTHE,16,Visa,6997.00,30602291487
2,2022-10-04,2022-11-01,2031,ARANCEL,144402,506,4547438,LE UTTHE,16,Mastercard,1895.97,30602291487
3,2022-10-04,2022-11-01,2031,ARANCEL,144450,506,25107174,LE UTTHE,16,Visa,125.95,30602291487
4,2022-10-04,2022-11-01,2800,TOTAL DEDUCCIONES,144402,506,4547438,LE UTTHE,16,Mastercard,9378.88,30602291487
...,...,...,...,...,...,...,...,...,...,...,...,...
47856,2022-11-29,2022-11-30,6000,IMPORTE NETO DE PAGOS,149349,506,25107594,LE UTTHE,16,Visa Debito,141642.98,30602291487
47857,2022-11-29,2022-11-30,6000,IMPORTE NETO DE PAGOS,149350,506,25107603,LE UTTHE,16,Visa Debito,190792.43,30602291487
47858,2022-11-29,2022-11-30,6000,IMPORTE NETO DE PAGOS,149351,506,25108471,LE UTTHE,16,Visa Debito,47190.30,30602291487
47859,2022-11-29,2022-11-30,6000,IMPORTE NETO DE PAGOS,149353,506,25892842,LE UTTHE,16,Visa Debito,304428.06,30602291487


In [7]:
#tarjetas = tarjetas.rename(columns=tarjetas.iloc[0]).drop(tarjetas.index[0]) #primera fila se transforma en Columnas
tarjetas.rename(columns={'Marca Producto': 'Cod Marca Producto'}, inplace=True)
tarjetas = tarjetas.drop(columns=['Fecha de Presentacion','Cod Concepto Clearing', 'Cod Sucursal Pagadora',
                  'Nombre Comercio', 'Cod Entidad Pagadora', 'Cuit Ruc Num']) #Borro columnas que no sirven
tarjetas = tarjetas.pivot_table(index=['Fecha de Pago', 'Cod Marca Producto','Cod Comercio', 'Nro Liquidacion'],
                     columns='Desc Concepto Clearing',
                     values='Importe Concepto',) 
#reordeno la tabla asignando nuevas columnas, las cuales provienen originalmente de "Desc Concepto Clearing"
tarjetas = tarjetas.reset_index() #reinicio el indice para "acomodar" columnas
tarjetas = tarjetas.rename(columns=lambda x: x.strip()) #elimino espacios en blanco de los extremos de los nombres de las columnas
tarjetas = tarjetas.drop(columns=['CRED FISCAL AHORA 12/18 VTA EN CUOTAS 10,50%',
       'CRED FISCAL AHORA 12/18 VTA EN CUOTAS 21,00%',
       'CRED. FISCAL PAG.S/DTO.FIN.OTORG 21,00%',
       'CRED.FISC.LEY 25063 PAG.S/DTO.F.OTORG. 10,50',
       'SUBTOTAL NETO DE PAGOS', 'TOTAL LIQUIDACION'
       ]) #Elimino las columnas de "Cred Fiscal" y "netos de pagos" 
tarjetas = tarjetas.fillna(0) # reemplazo todas las celdas Nan con un valor (0)
tarjetas = tarjetas.astype({'Cod Comercio':'int', 'Nro Liquidacion':'int'}) #transformo en int 2 columnas, las cuales no deben llevar decimales
%store tarjetas

Stored 'tarjetas' (DataFrame)


In [8]:
tarjetas.columns

Index(['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion', 'PERCEPCION IVA R.G. 2408   3,00 %',
       'AJUSTE SIRTAC NOV. 1RA QU', 'AJUSTE SIRTAC OCT. 2DA QUIN', 'ARANCEL',
       'CARGO  SISTEMA CUOTAS MENS', 'CARGO LIQUIDACION ELECTR.',
       'CARGO POSNET WWW.FISERV.COM.AR', 'CARGOS DE ENTIDAD',
       'DESCUENTO FINANC.OTORG. CUOTAS', 'IMPORTE NETO DE PAGOS',
       'IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50',
       'IVA CRED.FISC.COMERCIO S/ARANC 21,00%',
       'IVA PROMO CUOTAS AHORA 12/18 - 10,50%',
       'IVA PROMO CUOTAS AHORA 12/18 - 21,00%',
       'IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG', 'IVA RI SERV.OPER. INT.',
       'IVA RI SIST CUOTAS', 'IVA RI. CARGO LIQUID. ELECT.',
       'PER B.A.I.BR.DN.01/04', 'PERC.I.B. CORRIENTES R.165/00',
       'PERCEP ING BR MISIONES R.G. 003/93',
       'PERCEPCION IVA R.G. 2408   1,50 %', 'PROMO CUOTAS AHORA 12/18',
       'QR  RETENCION GANANCIAS', 'QR  RETENCION IIBB BS. AS.',
       'QR  RETENCION

In [ ]:
bancos

In [ ]:
tarjetas['Cod Marca Producto'].unique()